In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
import os
import json

from fuzzy_sql.rnd_query import RND_QUERY
from fuzzy_sql.fuzzy_sql import *

In [2]:
#set directories
root_dir=Path('/home/samer/projects/fuzzy_sql')
metadata_dir=os.path.join(root_dir,'data/sdgd/processed/metadata')
db_path=os.path.join(root_dir,'db/sdgd.db')

In [3]:
#define input tables and metadata

tbl_names_lst=["C1"]

metadata_lst=[]
for tbl_name in tbl_names_lst:
    with open(os.path.join(metadata_dir,tbl_name+'.json'),'r') as f:
        metadata_lst.append(json.load(f))

In [4]:
#setup parameters
DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    # 'FILTER_TYPE':{'WHERE':0.5, 'AND':0.5}, #Use WHERE or AND with JOIN CLAUSE
    # 'JOIN_TYPE': {'JOIN':0.5, 'LEFT JOIN':0.5}
}

#connect to db
conn = sqlite3.connect(db_path)

self=RND_QUERY(conn, tbl_names_lst, metadata_lst,DFLT_PARAMS)

Metadata fot table C1 validated.


# Generate Random  queries 

In [5]:
#Single Aggregate

rnd_query=self.make_single_agg_query(agg_fntn=True)


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT education, relationship, occupation, native_country, COUNT(*), SUM(capital)  FROM C1  GROUP BY education, relationship, occupation, native_country
no_of_rows=3942
no_of_cols=6
product=23.652


SELECT education, relationship, occupation, native_country, COUNT(*), SUM(capital)  FROM C1  GROUP BY education, relationship, occupation, native_country


In [6]:
rnd_query['query_desc']

{'type': 'single_agg',
 'agg_fntn': ('SUM', 'capital'),
 'grpby_vars': ['education', 'relationship', 'occupation', 'native_country'],
 'from_tbl_name': 'C1',
 'join_tbl_name_lst': [],
 'sql': 'SELECT education, relationship, occupation, native_country, COUNT(*), SUM(capital)  FROM C1  GROUP BY education, relationship, occupation, native_country',
 'n_rows': 3942,
 'n_cols': 6}

In [7]:
# Twin Aggregate 

syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_agg_query(syn_tbl_name_lst, agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")



SELECT marital_status, relationship, workclass, occupation, education, income, native_country, COUNT(*), MIN(age)  FROM C1  GROUP BY marital_status, relationship, workclass, occupation, education, income, native_country
SELECT marital_status, relationship, workclass, occupation, education, income, native_country, COUNT(*), MIN(age)  FROM C1  GROUP BY marital_status, relationship, workclass, occupation, education, income, native_country
REAL:
no_of_rows=9142
no_of_cols=9
product=82.278


SELECT marital_status, relationship, workclass, occupation, education, income, native_country, COUNT(*), MIN(age)  FROM C1_syn_default_1  GROUP BY marital_status, relationship, workclass, occupation, education, income, native_country
SYN:
no_of_rows=10279
no_of_cols=9
product=92.511




In [8]:
rnd_query['query_desc']

{'type': 'twin_agg',
 'agg_fntn': ('MIN', 'age'),
 'grpby_vars': ['marital_status',
  'relationship',
  'workclass',
  'occupation',
  'education',
  'income',
  'native_country'],
 'from_tbl_name_real': 'C1',
 'join_tbl_name_lst_real': [],
 'sql_real': 'SELECT marital_status, relationship, workclass, occupation, education, income, native_country, COUNT(*), MIN(age)  FROM C1  GROUP BY marital_status, relationship, workclass, occupation, education, income, native_country',
 'n_cols_real': 9,
 'n_rows_real': 9142,
 'from_tbl_name_syn': 'C1_syn_default_1',
 'join_tbl_name_lst_syn': [],
 'sql_syn': 'SELECT marital_status, relationship, workclass, occupation, education, income, native_country, COUNT(*), MIN(age)  FROM C1_syn_default_1  GROUP BY marital_status, relationship, workclass, occupation, education, income, native_country',
 'n_cols_syn': 9,
 'n_rows_syn': 10279}

In [9]:
#Single Filter

# self.no_groupby_vars=2
# self.no_where_vars=2
# self.max_in_terms=2

rnd_query=self.make_single_fltr_query()


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT *  FROM C1  WHERE  NOT  sex <> 'Male' AND  occupation NOT IN ('Farming-fishing', 'Tech-support', 'Sales') AND  capital = 0 OR  workclass = 'Private' 
no_of_rows=40959
no_of_cols=13
product=532.467


SELECT *  FROM C1  WHERE  NOT  sex <> 'Male' AND  occupation NOT IN ('Farming-fishing', 'Tech-support', 'Sales') AND  capital = 0 OR  workclass = 'Private' 


In [10]:
rnd_query['query_desc']

{'type': 'single_fltr',
 'from_tbl_name': 'C1',
 'join_tbl_name_lst': [],
 'sql': "SELECT *  FROM C1  WHERE  NOT  sex <> 'Male' AND  occupation NOT IN ('Farming-fishing', 'Tech-support', 'Sales') AND  capital = 0 OR  workclass = 'Private' ",
 'n_rows': 40959,
 'n_cols': 13}

In [11]:
# Twin Filter 


syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_fltr_query(syn_tbl_name_lst)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")
print(rnd_query['query_desc']['sql_real'])
print("\n")
print(rnd_query['query_desc']['sql_syn'])

SELECT *  FROM C1  WHERE   age = 73 AND NOT  marital_status LIKE 'Widowed' AND NOT  relationship NOT LIKE 'Husband' AND  sex <> 'Male' OR  income = '>50K' AND  education = 'HS-grad' AND  fnlwgt BETWEEN 118605 AND 77313 AND  native_country NOT IN ('United-States', 'United-States', 'United-States') AND  occupation NOT LIKE 'Other-service' 
REAL:
no_of_rows=113
no_of_cols=13
product=1.469


SYN:
no_of_rows=89
no_of_cols=13
product=1.157


SELECT *  FROM C1  WHERE   age = 73 AND NOT  marital_status LIKE 'Widowed' AND NOT  relationship NOT LIKE 'Husband' AND  sex <> 'Male' OR  income = '>50K' AND  education = 'HS-grad' AND  fnlwgt BETWEEN 118605 AND 77313 AND  native_country NOT IN ('United-States', 'United-States', 'United-States') AND  occupation NOT LIKE 'Other-service' 


SELECT *  FROM C1_syn_default_1  WHERE   age = 73 AND NOT  marital_status LIKE 'Widowed' AND NOT  relationship NOT LIKE 'Husband' AND  sex <> 'Male' OR  income = '>50K' AND  education = 'HS-grad' AND  fnlwgt BETWEEN 11

In [12]:
rnd_query['query_desc']

{'type': 'twin_fltr',
 'from_tbl_name_real': 'C1',
 'join_tbl_name_lst_real': [],
 'sql_real': "SELECT *  FROM C1  WHERE   age = 73 AND NOT  marital_status LIKE 'Widowed' AND NOT  relationship NOT LIKE 'Husband' AND  sex <> 'Male' OR  income = '>50K' AND  education = 'HS-grad' AND  fnlwgt BETWEEN 118605 AND 77313 AND  native_country NOT IN ('United-States', 'United-States', 'United-States') AND  occupation NOT LIKE 'Other-service' ",
 'n_cols_real': 13,
 'n_rows_real': 113,
 'from_tbl_name_syn': 'C1_syn_default_1',
 'join_tbl_name_lst_syn': [],
 'sql_syn': "SELECT *  FROM C1_syn_default_1  WHERE   age = 73 AND NOT  marital_status LIKE 'Widowed' AND NOT  relationship NOT LIKE 'Husband' AND  sex <> 'Male' OR  income = '>50K' AND  education = 'HS-grad' AND  fnlwgt BETWEEN 118605 AND 77313 AND  native_country NOT IN ('United-States', 'United-States', 'United-States') AND  occupation NOT LIKE 'Other-service' ",
 'n_cols_syn': 13,
 'n_rows_syn': 89}

In [13]:
#Single Aggregate-Filter

rnd_query=self.make_single_aggfltr_query(agg_fntn=True)


n_rows=rnd_query['query_desc']['n_rows']
n_cols=rnd_query['query_desc']['n_cols']

print(f"no_of_rows={n_rows}\nno_of_cols={n_cols}\nproduct={n_rows*n_cols/1000}")
print("\n")
print(rnd_query['query_desc']['sql'])

SELECT sex, race, income, workclass, education, occupation, relationship, native_country, COUNT(*), SUM(capital)  FROM C1  WHERE   relationship LIKE 'Not-in-family' AND  native_country LIKE 'United-States' AND  workclass <> 'Self-emp-inc' AND  fnlwgt NOT BETWEEN 175697 AND 186932 AND NOT  marital_status LIKE 'Never-married' AND  education NOT LIKE 'HS-grad'  GROUP BY sex, race, income, workclass, education, occupation, relationship, native_country
no_of_rows=845
no_of_cols=10
product=8.45


SELECT sex, race, income, workclass, education, occupation, relationship, native_country, COUNT(*), SUM(capital)  FROM C1  WHERE   relationship LIKE 'Not-in-family' AND  native_country LIKE 'United-States' AND  workclass <> 'Self-emp-inc' AND  fnlwgt NOT BETWEEN 175697 AND 186932 AND NOT  marital_status LIKE 'Never-married' AND  education NOT LIKE 'HS-grad'  GROUP BY sex, race, income, workclass, education, occupation, relationship, native_country


In [14]:
# Twin Aggregate-Filter 

syn_tbl_name_lst=['C1_syn_default_1']
rnd_query=self.make_twin_aggfltr_query(syn_tbl_name_lst, agg_fntn=True)

n_rows_real=rnd_query['query_desc']['n_rows_real']
n_cols_real=rnd_query['query_desc']['n_cols_real']
n_rows_syn=rnd_query['query_desc']['n_rows_syn']
n_cols_syn=rnd_query['query_desc']['n_cols_syn']

print(rnd_query['query_desc']['sql_real'])
print(f"REAL:\nno_of_rows={n_rows_real}\nno_of_cols={n_cols_real}\nproduct={n_rows_real*n_cols_real/1000}\n\n")

print(rnd_query['query_desc']['sql_syn'])
print(f"SYN:\nno_of_rows={n_rows_syn}\nno_of_cols={n_cols_syn}\nproduct={n_rows_syn*n_cols_syn/1000}\n\n")

SELECT workclass, marital_status, sex, race, income, native_country, COUNT(*), SUM(fnlwgt)  FROM C1  WHERE   education = '10th' AND  income <> '<=50K' OR  workclass NOT LIKE 'Self-emp-not-inc' AND  age = 22 AND NOT  native_country = 'United-States' AND  relationship <> 'Other-relative' AND NOT  marital_status IN ('Divorced', 'Married-civ-spouse', 'Married-civ-spouse') AND  hours_per_week NOT BETWEEN 40 AND 60 AND NOT  occupation IN ('Machine-op-inspct', 'Machine-op-inspct', 'Prof-specialty') AND  race = 'White' AND  sex = 'Female' AND  capital NOT BETWEEN 0 AND 0  GROUP BY workclass, marital_status, sex, race, income, native_country
SELECT workclass, marital_status, sex, race, income, native_country, COUNT(*), SUM(fnlwgt)  FROM C1  WHERE   education = '10th' AND  income <> '<=50K' OR  workclass NOT LIKE 'Self-emp-not-inc' AND  age = 22 AND NOT  native_country = 'United-States' AND  relationship <> 'Other-relative' AND NOT  marital_status IN ('Divorced', 'Married-civ-spouse', 'Married-c